# Project 2: Forest Fire Simulation

In this project, you will simulate the spread of a forest fire on a 100×100 grid. Each cell in the grid can be in one of four states: tree, grassland, burning tree, or burnt tree. The goal is to understand how forest density influences fire spread, and to explore extensions such as the effect of wind.

## Part 1: Initial Setup of the Grid
- Grid and States:
- Create a 100×100 NumPy array to represent the forest.
- Each cell can have one of the following values:
	0: Grassland (non-flammable)
	1: Tree
	2: Burning tree
	3: Burnt tree
- Random Initialization: 
	- Implement a function that initializes the grid with trees and grassland based on a tree density parameter (between 0 and 1).
	- Each cell should be a tree with probability = density, and grassland otherwise.
Lightning Strike:
	- Randomly select a cell that contains a tree (1) and set it to burning (2).

## Part 2: Fire Spread Simulation
- Simulation Step:
	- In each time step:
		- A burning tree (2) becomes a burnt tree (3).
		- All trees (1) that are directly adjacent (up, down, left, right) to a burning tree catch fire and become burning trees (2) in the next step.
- Run the Simulation:
	-Continue the simulation step-by-step until no more trees are burning.
- Track Results:
	- Record the number of trees burnt at the end of the simulation.
- Calculate the percentage of trees burnt compared to the initial number of trees.


## Part 3: Visualization and Analysis
- Graphical Representation:
	- Write a function that can visualize the grid at each time step by using matplotlib. Use different colors for each state.
	- Write a function that allows you to created an animated gif for a full run of the simulation, where each frame/picture corresponds to a time-step
- Density Curve:
	- Run the simulation for various density values (e.g., from 0.1 to 1.0 in steps of 0.05). Clearly you need to run the simulation several times for each density value and take then the mean value of the resulting percentage of trees burnt.
	- Plot the percentage of trees burnt as a function of the initial density including the statistical uncertainties.
- Critical Density:
	- From the plot, identify the critical density above which the fire spreads through almost the entire forest.
- Expert Challenge (not strictly required to be handed in, however is required for achieving the best mark): Larger grids, e.g. 1000x1000 require in a naive simulation significantly more time, hence it is advisable to think about certain optimization aspects. One promising approach is a clever usage of numpy arrays and slicing. You can add one additional "optimizedSimulation" function that can handle efficiently also larger map sizes and is more time-effective than a naive approach that directly loops on your arrays.

## Part 4: Extensions – Wind Effect
- Wind Influence:
	- Modify the fire-spread rule to account for wind blowing in one direction (e.g., east). For example, a tree that is east of a burning tree can catch fire even if it is not only the direct next neighbour. The strength of the wind might determine the spread radius in one direction. It is important that you come up here with your own model of how wind might effect the spread of fire. It is your task for explain the choice of your model. Think independently! As long as your model choice is sensible, you will get full points. 
Discuss:
	- What impact does wind have on the spread of the fire in your model?
	- How does the critical density change when wind is included?

## Deliverables:
- A Jupyter-Notebook implementing the simulation, including
	- A plot showing burnt tree percentage vs. initial density.
	- One Animation of the fire spreading at a given density and one lighting event.
	- A brief written summary of findings and observations about the critical density and wind effects.

## Task 1

In [22]:
# import needed modules
import numpy as np 
import matplotlib as mpl
from matplotlib import pyplot as plt 
from matplotlib.colors import ListedColormap
import imageio.v2 as imageio
import os
import glob


In [23]:
## legend
GRASSLAND = 0
TREE = 1
BURNING = 2
BURNT = 3

size = 100
forest = np.zeros([size,size])


def fill_forest(density):
    '''Creates an array of specified size with the probability for the trees being specified by the density. There is one burning tree due to a lightning strike'''

    # fill the forest with trees depending on the density
    forest[np.random.rand(size,size) < density] = 1

    # Select a random value from the forest array and set it to 2, to indicate a burning tree
    lightning_position = np.random.choice(size),np.random.choice(size)
    forest[lightning_position] = 2

fill_forest(0.7)


In [24]:
forest

array([[1., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 1., 1., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 1.],
       [1., 1., 0., ..., 1., 1., 0.],
       [1., 1., 1., ..., 0., 1., 0.]])

## Task 2

In [25]:
def simulate_fire(forest, output=True):
    ''' A function that  takes a forest grid and simulates the spreading of the forest fire if at least one tree is burning'''
    
    # get the grid size of of the forest
    grid_size = forest.shape[0] 
    # make a copy of the current forest                                                    
    history = [forest.copy()]
    
    # create a copy of the forest as long as there are any burning trees
    while np.any(forest == BURNING):                                               
        new_forest = forest.copy()                                          


        # loop for axis 0 of the forest grid
        for i in range(grid_size):                                                  
            # loop for axis 1 of the forest grid
            for j in range(grid_size):                                              
                if forest[i, j] == BURNING:
                    # in this step, set the burning trees to burnt trees 
                    new_forest[i, j] = BURNT                                        
                    # define dx amd dy as the steps to the neighboring pixels
                    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:               
                        # define the indices of the neighboring pixels ni and nj
                        ni, nj = i + dx, j + dy                                     
                        # loop over the pixels as long as they are in the grid
                        if 0 <= ni < grid_size and 0 <= nj < grid_size:             
                            # check for all pixels if the neighboring pixels are trees
                            if forest[ni, nj] == TREE:                              
                                # if this is the case, set those pixels from Tree to burning tree so from 1 to 2
                                new_forest[ni, nj] = BURNING                        


        # update the current forest grid
        forest = new_forest                                                         
        # add the current forest grid to the history, for the simulation   
        history.append(forest.copy())                                               
        # calculate the number of trees burned after the fire
        burnt_trees = np.sum(history[-1] == 3)  
        # total number of trees at the start
        trees_at_start = np.sum(history[0] == 1)+1     
        # calculate the number of trees burned after the fire
        burnt_percentage =  np.sum(history[-1] == 3)/(np.sum(history[0] == 1)+1) * 100   
        # return final_forest 
        final_forest = history[-1]

    # return all the forest grids during the simulation, the total number of burned tree in the and, and the percentage of trees burnt compared to the initial number of trees.
    if output == True:
        print(f'The total number of burned trees is {burnt_trees}. Before the fire there were {trees_at_start} trees.')
        print(f'Therefore {burnt_percentage:.2f}% of the initial number of trees are burnt after the fire.')

    return history  #final_forest, burnt_percentage, burnt_trees, trees_at_start

In [26]:
forest = simulate_fire(forest)


The total number of burned trees is 6697. Before the fire there were 6947 trees.
Therefore 96.40% of the initial number of trees are burnt after the fire.


## Task 3

In [27]:
def plots(forest, save):
    # remove frames from previous simulations if new frames are to be saved
    if save == True:
        files_to_remove = np.sort(glob.glob("frames/frame_*.png"))
        for file in files_to_remove:
            os.remove(file)

    # find the amount of steps the fire simulation took
    z = np.shape(forest)[0]
    print(z)
    i = 0
    # loop over each step for a fire simulation 
    while i < z:
        # create a figure
        fig, ax = plt.subplots(figsize=(8,7))
        
        # create colormap
        bounds = [0, 1, 2, 3, 4]
        cmap = mpl.colors.ListedColormap(['lightgreen', 'darkgreen', 'red', 'gray'])
        norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

        # remove labels from the label ticks x and y axis
        ax.set_xticks([])
        ax.set_yticks([])

        # plot the forest grid
        plt.imshow(forest[i], cmap=cmap,  norm=norm)
        
        # create the colorbar
        cbar = plt.colorbar(ticks=[0.5, 1.5, 2.5, 3.5], shrink=0.9)
        
        # set the legend for the colorbar
        cbar.ax.set_yticklabels(['Grassland', 'Tree', 'Burning tree', 'Burnt tree'])
        
        # give each plot a name for saving it later
        frame = i

        if save == True: 
            plt.savefig(f"frames/frame_{frame}.png", bbox_inches='tight')
            plt.close(fig)
        else:
            plt.show()

        i = i+1
        
plots(forest, True)

172


In [28]:
def create_gif():
    # get the files into a list
    filenames = glob.glob("frames/frame_*.png")
    
    # Extract the numeric part from each filename and convert to int
    numbers = [int(f.split('_')[1].split('.')[0]) for f in filenames]
    
    # Find the maximum value
    max_x = np.max(numbers)
    
    #create a gif
    n = 0
    with imageio.get_writer('frames/forest_fire.gif', mode='I', duration=0.1) as writer:
        while n <= max_x:
            image = imageio.imread(f'frames/frame_{n}.png')
            writer.append_data(image)
            n = n +1
            #os.remove(f"frames/frame_{frame}.png")

In [29]:
create_gif()

In [30]:
# remove frames a gain
files_to_remove = np.sort(glob.glob("frames/frame_*.png"))
for file in files_to_remove:
    os.remove(file)